In [81]:
#import requirements

import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score


In [82]:
#denver data from arcGIS

df = pd.read_csv('/Users/andrewkerr/Downloads/denver.csv')
df

/var/folders/4x/12hsw44n14sgp38ys227k7qm0000gn/T/ipykernel_33397/198947694.py:1: DtypeWarning: Columns (33,53,55,58,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/andrewkerr/Downloads/denver.csv')


,Unnamed: 0,prem_full_addr,OBJECTID,lsli_id,objectid_1,tap_number,gismaininstalldate,gismaininstallyear,gisabanmaininstalldate,gisabanmaininstallyear,...,Prem_id_current,tap_number_current,task_order_area,NHID,NHNAME,CRITICAL_CUSTOMER_FLAG,Category_Desc,Length,Premise_ID,GlobalID
0,0,7370 E FLORIDA AVE,1,523264,137272,287796.0,4.456512e+11,1984.0,NaN,0.0,...,8.837921e+09,287796,NaN,38,Indian Creek,0,Based on your property’s construction date and...,10.0,8.837921e+09,80a66794-2b34-4783-a1a2-bdcc7f9bf26d
1,3,5592 W PRENTICE CIR,4,523267,143910,326333.0,8.760960e+11,1997.0,NaN,0.0,...,9.433841e+09,326333,NaN,44,Marston,0,Based on your property’s construction date and...,10.0,9.433841e+09,9673cfa1-161f-4669-93e4-5dd5a5515eb7
2,4,5466 W PRENTICE CIR,5,523268,108119,326351.0,8.760960e+11,1997.0,NaN,0.0,...,6.146840e+09,326351,NaN,44,Marston,0,Based on your property’s construction date and...,10.0,6.146840e+09,f85d08c4-b513-4d63-9642-64a672ff7a45
3,5,5460 W PRENTICE CIR,6,523269,79872,326352.0,8.760960e+11,1997.0,NaN,0.0,...,3.533841e+09,326352,NaN,44,Marston,0,Based on your property’s construction date and...,10.0,3.533841e+09,15a368c0-bf90-47fa-9fc2-61cda2135206
4,6,565 W JAMISON CIR,7,523270,46966,288476.0,4.534272e+11,1984.0,NaN,0.0,...,5.105405e+08,288476,NaN,276,South Littleton,0,Based on your property’s construction date and...,9.0,5.105405e+08,87bbaa5c-a999-4030-a1b1-54cd280a3e09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193493,320616,4430 COLUMBINE ST,335440,670937,335480,389864.0,NaN,NaN,NaN,NaN,...,2.204684e+08,389864,2021 GEO-WAQ-TO2,25,Elyria Swansea,0,Based on your property’s construction date and...,9.0,2.204684e+08,0b692c3b-59cf-4083-a6b2-e825bf4f0219
193494,320617,4480 PENNSYLVANIA ST,335441,670925,335468,388942.0,NaN,NaN,NaN,NaN,...,4.120832e+09,388942,2021 GEO-WAR-TO1,29,Globeville,0,Based on your property’s construction date and...,10.0,4.120832e+09,6844d51b-5c19-40a6-b919-deb54a2b070d
193495,320618,1444 LIPAN ST,335442,670945,335488,390447.0,NaN,NaN,NaN,NaN,...,7.450413e+09,390447,NaN,41,Lincoln Park,0,Based on your property’s construction date and...,10.0,7.450413e+09,a2efc6d0-206a-48e7-996a-78808efed16e
193496,320619,1458 KNOX CT,335443,670946,335489,390566.0,NaN,NaN,NaN,NaN,...,1.112047e+09,390566,NaN,74,West Colfax,0,Based on your property’s construction date and...,10.0,1.112047e+09,e0745d64-034d-4792-ab9e-dee3525ddacf


In [83]:
#choosing relevant columns for modeling

df = df[['parcelyearbuilt', 'tapsize', 'servicelinesize', 'servicearea', 'mediumhouseholdincome2015', 'point_x', 'point_y', 'date_replaced', 'denver_district', 'lsli_final_lead_cat', 'NHID', 'NHNAME', 'Length']]
df

,parcelyearbuilt,tapsize,servicelinesize,servicearea,mediumhouseholdincome2015,point_x,point_y,date_replaced,denver_district,lsli_final_lead_cat,NHID,NHNAME,Length
0,1984.0,1,1,INSIDE CITY,71099.0,3168369.912,1676422.397,NaN,6.0,No lead service line.,38,Indian Creek,10.0
1,1998.0,4-Mar,4-Mar,INSIDE CITY,112490.0,3123891.219,1650617.053,NaN,2.0,No lead service line.,44,Marston,10.0
2,1999.0,4-Mar,4-Mar,INSIDE CITY,112490.0,3124677.052,1650668.561,NaN,2.0,No lead service line.,44,Marston,10.0
3,1998.0,4-Mar,4-Mar,INSIDE CITY,112490.0,3124639.144,1650591.668,NaN,2.0,No lead service line.,44,Marston,10.0
4,1984.0,4-Mar,4-Mar,TOTAL SERVICE,101286.0,3142223.133,1635854.373,NaN,NaN,No lead service line.,276,South Littleton,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
193493,1933.0,4-Mar,4-Mar,INSIDE CITY,NaN,3152631.270,1709773.170,NaN,NaN,No lead service line.,25,Elyria Swansea,9.0
193494,1949.0,4-Mar,4-Mar,INSIDE CITY,NaN,3145918.430,1708714.220,NaN,NaN,No lead service line.,29,Globeville,10.0
193495,1909.0,4-Mar,4-Mar,INSIDE CITY,NaN,3140255.750,1694554.500,NaN,NaN,No lead service line.,41,Lincoln Park,10.0
193496,1909.0,4-Mar,4-Mar,INSIDE CITY,NaN,3131620.000,1694603.500,NaN,NaN,No lead service line.,74,West Colfax,10.0


In [84]:
#checking number of lead/no lead counts

df['lsli_final_lead_cat'].value_counts()

lsli_final_lead_cat
No lead service line.                                            171185
Confirmed lead service line; included in replacement program.     22313
Name: count, dtype: int64

In [85]:
#feature engineering on lead columns

df['lsli_final_lead_cat'] = df['lsli_final_lead_cat'].replace({'No lead service line.': 0, 'Confirmed lead service line; included in replacement program.': 1})
df

/var/folders/4x/12hsw44n14sgp38ys227k7qm0000gn/T/ipykernel_33397/2020141344.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lsli_final_lead_cat'] = df['lsli_final_lead_cat'].replace({'No lead service line.': 0, 'Confirmed lead service line; included in replacement program.': 1})


,parcelyearbuilt,tapsize,servicelinesize,servicearea,mediumhouseholdincome2015,point_x,point_y,date_replaced,denver_district,lsli_final_lead_cat,NHID,NHNAME,Length
0,1984.0,1,1,INSIDE CITY,71099.0,3168369.912,1676422.397,NaN,6.0,0,38,Indian Creek,10.0
1,1998.0,4-Mar,4-Mar,INSIDE CITY,112490.0,3123891.219,1650617.053,NaN,2.0,0,44,Marston,10.0
2,1999.0,4-Mar,4-Mar,INSIDE CITY,112490.0,3124677.052,1650668.561,NaN,2.0,0,44,Marston,10.0
3,1998.0,4-Mar,4-Mar,INSIDE CITY,112490.0,3124639.144,1650591.668,NaN,2.0,0,44,Marston,10.0
4,1984.0,4-Mar,4-Mar,TOTAL SERVICE,101286.0,3142223.133,1635854.373,NaN,NaN,0,276,South Littleton,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
193493,1933.0,4-Mar,4-Mar,INSIDE CITY,NaN,3152631.270,1709773.170,NaN,NaN,0,25,Elyria Swansea,9.0
193494,1949.0,4-Mar,4-Mar,INSIDE CITY,NaN,3145918.430,1708714.220,NaN,NaN,0,29,Globeville,10.0
193495,1909.0,4-Mar,4-Mar,INSIDE CITY,NaN,3140255.750,1694554.500,NaN,NaN,0,41,Lincoln Park,10.0
193496,1909.0,4-Mar,4-Mar,INSIDE CITY,NaN,3131620.000,1694603.500,NaN,NaN,0,74,West Colfax,10.0


In [86]:
#feature engineering to trasnform categorical -> numerical data

label_encoder = LabelEncoder()
df['tapsize'] = label_encoder.fit_transform(df['tapsize'])
df['servicelinesize'] = label_encoder.fit_transform(df['servicelinesize'])
df['servicearea'] = label_encoder.fit_transform(df['servicearea'])
df['date_replaced'] = label_encoder.fit_transform(df['date_replaced'])
df['NHNAME'] = label_encoder.fit_transform(df['NHNAME'])
df

/var/folders/4x/12hsw44n14sgp38ys227k7qm0000gn/T/ipykernel_33397/746400929.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tapsize'] = label_encoder.fit_transform(df['tapsize'])
/var/folders/4x/12hsw44n14sgp38ys227k7qm0000gn/T/ipykernel_33397/746400929.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['servicelinesize'] = label_encoder.fit_transform(df['servicelinesize'])
/var/folders/4x/12hsw44n14sgp38ys227k7qm0000gn/T/ipykernel_33397/746400929.py:4: SettingWithCopyWarning: 
A value is trying 

,parcelyearbuilt,tapsize,servicelinesize,servicearea,mediumhouseholdincome2015,point_x,point_y,date_replaced,denver_district,lsli_final_lead_cat,NHID,NHNAME,Length
0,1984.0,0,0,1,71099.0,3168369.912,1676422.397,675,6.0,0,38,67,10.0
1,1998.0,12,13,1,112490.0,3123891.219,1650617.053,675,2.0,0,44,75,10.0
2,1999.0,12,13,1,112490.0,3124677.052,1650668.561,675,2.0,0,44,75,10.0
3,1998.0,12,13,1,112490.0,3124639.144,1650591.668,675,2.0,0,44,75,10.0
4,1984.0,12,13,4,101286.0,3142223.133,1635854.373,675,NaN,0,276,103,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
193493,1933.0,12,13,1,NaN,3152631.270,1709773.170,675,NaN,0,25,39,9.0
193494,1949.0,12,13,1,NaN,3145918.430,1708714.220,675,NaN,0,29,47,10.0
193495,1909.0,12,13,1,NaN,3140255.750,1694554.500,675,NaN,0,41,71,10.0
193496,1909.0,12,13,1,NaN,3131620.000,1694603.500,675,NaN,0,74,132,10.0


In [88]:
#MODELING

X = df.drop(columns = 'lsli_final_lead_cat')
y = df['lsli_final_lead_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [89]:
bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
# fit model
bst.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed 

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=2, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [90]:
preds = bst.predict(X_test)
preds

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed 

array([0, 0, 1, ..., 0, 0, 1])

In [91]:
accuracy = accuracy_score(y_test, preds)
accuracy

0.9417571059431524